### We want to predict the protein seq



In [114]:
import more_itertools as mit
import itertools as it
import sys,os,time
import requests
from lxml import etree
from time import time
from bs4 import BeautifulSoup

class SeqPrediction():
    
    def __init__(self):
        self._target = '' # for now, change later
        self._permu = None # maybe not
        #Oc1 Need to modify
        self._left =''
        self._right = ''
        ## others
        self._protein =''
        self._spec =''
        self._start = 0
        self._stop = 0
        
        
        
    def set_seq(self,protein, spec, start,stop):
        """ set which protein to predict
        example, protein = 'Oc1', spec = 'mouse'
        start = 10
        stop 40   
        """
        if protein == 'Oc1' and spec == 'mouse':
            temp = """ MNAQLTMEAIGELHGVSHEPVPAPADLLGGSPHARSSVGHRGSHLPPAHPRSMGMASLLD
GGSGGSDYHHHHRAPEHSLAGPLHPTMTMACETPPGMSMPTTYTTLTPLQPLPPISTVSD
KFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDERGLASMNNLYTPYHKDVAGMGQSLS
PLSGSGLGSIHNSQQGLPHYAHPGAAMPTDKMLTPNGFEAHHPAMLGRHGEQHLTPTSAG
MVPINGLPPHHPHAHLNAQGHGQLLGTAREPNPSVTGAQVSNGSNSGQMEEINTKEVAQR
ITTELKRYSIPQAIFAQRVLCRSQGTLSDLLRNPKPWSKLKSGRETFRRMWKWLQEPEFQ
RMSALRLAACKRKEQEHGKDRGNTPKKPRLVFTDVQRRTLHAIFKENKRPSKELQITISQ
QLGLELSTVSNFFMNARRRSLDKWQDEGGSNSGSSSSSSSTCTKA"""
            temp ="".join(temp.split())      
        else:
            print('need to implement the other seq')
        # now get the target fragment
        self._target = temp[start:stop+1]
        self._left = temp[:start]
        self._right = temp[stop+1:]
        self._protein = protein
        self._spec =spec
   
    def validate(self,verbose = False):
        """ confirm the seq has be setted """
        # output so the user will know it is correct
        if self._left and self._right and self._target:
            total = "".join([self._left, self._right,self._target])
            if verbose:
                self.show_seq(total,self._target,'wild type seq')
            return True
        else:
            if verbose:
                print('setting your seq please')
            return False
           
        
    def _mutate(self):
        """
        In the mutate I will have iteratable to generate 
        """
        self._permu = mit.distinct_permutations(list(self._target))
    
    
    def generate_seq(self):
        """
        A generator, user can access
        Discuss with Muareen, what is the other condition we want to added
        If not combine left and right and produce the final results.
        """
        self._mutate()
        try:
            for frag in self._permu:
                frag ="".join(frag)
                total = self._left + frag + self._right
                yield  total, frag
                
        except StopIteration:
            raise StopIteration('finished all the possible elements already!')
    
    def show_seq(self,total,seq_fragment, category):
        print("""  
                You are going to analyze: {}, the {} seq is:\n\n{}***{}***{}\n
                it has total {} aa,
                the region you want to mutant is:{}   
                """.format(self._protein, category,self._left,seq_fragment, self._right,len(total), seq_fragment))
            
        
        
    def _query(self,oc1):
        """
        search one record
        """
        url = 'https://disorderatlas.med.umich.edu/protein_analysis/'
        client = requests.session()
        tree = etree.HTML(client.get(url).content)
        csrf = tree.xpath('//input[@name="csrfmiddlewaretoken"]/@value')[0]
        headers = {'referer': url, 'content-type': 'application/x-www-form-urlencoded', 'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
            # need to be replaced
        
        # query for one sequence
        values = {'csrfmiddlewaretoken' : csrf,
                  'seq_field': oc1,
                  'organism': 'Mus musculus' }

        r = client.post(url, data=values, headers=headers)
        soup = BeautifulSoup(r.content)
        oc1_table = soup.findAll('table',{'class':'table table-hover'})
        return oc1_table
    
    def _organize(self):
        # discuss with Maureen what do we want 
        """
        retrive the value we want from the table.
        """
        
    
    def excute(self,n): # this is the only method all 
        # Discuss with Maureen, whether we will want to have all the results
        # or we only want to have the big or small....
        
        """
        this method will query the website and save the results
        
        """
        pass
    
    def save_prediction(seqs, disorder_score):
        ## Need to be modified, We query on and save one, or we can do batches. All depending on what we really need.
        output_dir="."
        out_file='result.txt'
        save_file = os.path.join(output_dir, out_file)
        with open(save_file,mode = 'a+') as file:
            file.seek(2) # go to end
            for seq,disorder in zip(seqs, disorder_score):
                prediction = seq[1] +','+str(disorder) +'\n'
                file.writelines(prediction)
                
    def __iter__(self):
        """ 
        I am not sure whether we want this right now.
        If yes, I will yield the as tuple(permuation, value)     
        """
        pass



In [110]:
import time
a = SeqPrediction()
a.set_seq('Oc1','mouse',370,384)
a.validate(verbose = True)
    
### Important!!!! Change generate_seq to a non public method!!!!
### users should not access it directly!!!
### Do not forget !!!
t = a.generate_seq()
results =[]
for i in range(3):
    result = next(t)
    results.append(result)
    a.show_seq(result[0],result[1],'MUTANT')
    print(list(result[1]))
    
seq1 = results[1][0]
seq2 = results[2][0]
print('\nthe first seq', seq1)
print('\nthe second seq',seq2)
seq3 = seq2[1:100]
print('\n very short',seq3)
t1 = a._query(seq1)
time.sleep(0.03)
t2 = a._query(seq2)
time.sleep(0.03)
t3 = a._query(seq3)
time.sleep(0.03)


    

  
                You are going to analyze: Oc1, the wild type seq seq is:

MNAQLTMEAIGELHGVSHEPVPAPADLLGGSPHARSSVGHRGSHLPPAHPRSMGMASLLDGGSGGSDYHHHHRAPEHSLAGPLHPTMTMACETPPGMSMPTTYTTLTPLQPLPPISTVSDKFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDERGLASMNNLYTPYHKDVAGMGQSLSPLSGSGLGSIHNSQQGLPHYAHPGAAMPTDKMLTPNGFEAHHPAMLGRHGEQHLTPTSAGMVPINGLPPHHPHAHLNAQGHGQLLGTAREPNPSVTGAQVSNGSNSGQMEEINTKEVAQRITTELKRYSIPQAIFAQRVLCRSQGTLSDLLRNPKPWSKLKSGRETFRRMWKWLQEPEFQRMSALRLAAC***KRKEQEHGKDRGNTP***KKPRLVFTDVQRRTLHAIFKENKRPSKELQITISQQLGLELSTVSNFFMNARRRSLDKWQDEGGSNSGSSSSSSSTCTKA

                it has total 465 aa,
                the region you want to mutant is:KRKEQEHGKDRGNTP   
                
  
                You are going to analyze: Oc1, the MUTANT seq is:

MNAQLTMEAIGELHGVSHEPVPAPADLLGGSPHARSSVGHRGSHLPPAHPRSMGMASLLDGGSGGSDYHHHHRAPEHSLAGPLHPTMTMACETPPGMSMPTTYTTLTPLQPLPPISTVSDKFPHHHHHHHHHHHPHHHQRLAGNVSGSFTLMRDERGLASMNNLYTPYHKDVAGMGQSLSPLSGSGLGSIHNSQQGLPHYAHPGAAMPTDKMLTPNGFEAHHPAMLGRHGEQHLTPTSAGMVPINGLPPHHPHAHLNA

In [109]:
def save_prediction(seqs, disorder_score):
    output_dir="."
    out_file='result.txt'
    save_file = os.path.join(output_dir, out_file)
    with open(save_file,mode = 'a+') as file:
        file.seek(2) # go to end
        for seq,disorder in zip(seqs, disorder_score):
            print(seq[1],disorder)
            prediction = seq[1] +','+str(disorder) +'\n'
            file.writelines(prediction)

In [108]:
score =list(range(3))
save_prediction(results,score)
print(score)

PTNGRDKGHEQEKRK 0
TPNGRDKGHEQEKRK 1
TNPGRDKGHEQEKRK 2
[0, 1, 2]


In [111]:
print(t3[1])

<table class="table table-hover">
<thead>
<tr>
<th></th>
<th>IUPred</th>
<th>DisEMBL - H</th>
<th>DisEMBL - R</th>
<th>Consensus</th>
</tr>
</thead>
<tbody>
<tr>
<td>Value</td>
<td>100.0 %</td>
<td>30.3 %</td>
<td>33.3 %</td>
<td>6.1 %</td>
</tr>
<tr>
<td>Percentile</td>
<td>99.0</td>
<td>44.0</td>
<td>85.0</td>
<td>57.0</td>
</tr>
</tbody>
</table>


In [112]:
print(t3[0])

<table class="table table-hover">
<tbody>
<tr>
<td><strong>Protein ID:</strong></td>
<td><strong>Unknown_protein</strong></td>
</tr>
<tr>
<td><strong>Proteome:</strong></td>
<td><i>Mus musculus</i></td>
</tr>
<tr>
<td><strong>Protein length:</strong></td>
<td>99 aa</td>
</tr>
<tr>
<td><strong>Mean hydropathy:</strong><br/> (calculated from a skewed Kyte &amp; Doolittle hydrophobicity scale)</td>
<td>4.04343</td>
</tr>
<tr>
<td><strong>Uversky hydropathy:</strong><br/> (calculated from a normalized Kyte &amp; Doolittle hydrophobicity scale)</td>
<td>0.44927</td>
</tr>
<tr>
<td><strong>Fraction of charged residues:</strong></td>
<td>0.12121</td>
</tr>
<tr>
<td><strong>Net charge per residue:</strong></td>
<td>-0.0404</td>
</tr>
<tr>
<td><strong>Proline content:</strong></td>
<td>13.13</td>
</tr>
<tr>
<td><strong>Kappa:</strong></td>
<td>0.22621</td>
</tr>
</tbody>
</table>
